In [2]:
import pandas as pd
import requests

## Sleeper API Base Functions

In [ ]:
def get_nfl_state():
    url = "https://api.sleeper.app/v1/state/nfl"
    return requests.get(url).json()

In [117]:
def get_players() -> dict:
    url = "https://api.sleeper.app/v1/players/nfl"
    return requests.get(url).json()

In [ ]:
def get_user_id(username: str) -> str:
    url = f"https://api.sleeper.app/v1/user/{username}"
    return requests.get(url).json()["user_id"]

In [127]:
def get_leagues(user_id: str, season: str) -> dict:
    url = f"https://api.sleeper.app/v1/user/{user_id}/leagues/nfl/{season}"
    return requests.get(url).json()

In [ ]:
def get_league_info(league_id: str) -> dict:
    url = f"https://api.sleeper.app/v1/league/{league_id}"
    return requests.get(url).json()

In [ ]:
def get_users(league_id: str) -> dict:
    url = f"https://api.sleeper.app/v1/league/{league_id}/users"
    return requests.get(url).json()

In [ ]:
def get_matchups(league_id: str, week: int) -> dict:
    url = f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week}"
    return requests.get(url).json()

In [ ]:
def get_rosters(league_id: str) -> dict:
    url = f"https://api.sleeper.app/v1/league/{league_id}/rosters"
    return requests.get(url).json()

In [ ]:
def get_winners_bracket(league_id: str) -> dict:
    url = f"https://api.sleeper.app/v1/league/{league_id}/winners_bracket"
    return requests.get(url).json()

## Main Functions

In [122]:
def players_df(columns_to_keep) -> pd.DataFrame:
    
    # Get all the player data from the base fxn
    players = get_players()

    # Build a list of dicts with only the columns you want
    extracted_list = []
    for player_id, player_info in players.items():
        extracted_list.append({col: player_info.get(col) for col in columns_to_keep})

    # Create DataFrame from the list
    df_players = pd.DataFrame(extracted_list)

    # Keep only players with a team assigned to them
    df_players = df_players[df_players['team'].notna()].copy()

    return df_players

In [156]:
def list_league_ids(user_id: str, season: str) -> list:
    leagues = get_leagues(user_id, season)  # assuming get_leagues takes these args
    return [l["league_id"] for l in leagues]

In [261]:
def user_dictionary(league_id: str) -> dict:
    user_data = get_users(league_id)

    # users_json is your original list of user dictionaries
    user_dict = {}

    for user in user_data:
        metadata = user.get('metadata', {})
        user_dict[user['user_id']] = {
            'display_name': user.get('display_name'),
            'team_name': metadata.get('team_name'),
            'avatar': metadata.get('avatar')
        }

    return user_dict

In [ ]:
def get_winner_user_id(league_id:str) -> str:
    data = get_winners_bracket(league_id)

    target = next((item for item in data if item.get('m') == 6), None)

    if target:
        w_value = target.get('w')

    mapping = get_owner_to_roster_map(league_id)

    winner_user_id = mapping.get(w_value, None)

    return winner_user_id 

In [403]:
def league_info_all_years_df(current_league_id: str) -> pd.DataFrame:
    rows = []


    league_id = current_league_id

    while league_id:

        league_info = get_league_info(league_id)

        # Instead of listing fields, dynamically take all top-level keys
        # and just keep their raw values (nested dicts stay intact)
        row = {k: v for k, v in league_info.items()}

        # Add the users dictionary as a new column
        row['users'] = user_dictionary(league_id)


        row['reg_season_champ'] = get_winner_user_id(league_id)

        row['playoff_data'] = get_winners_bracket(league_id)

        rows.append(row)

        league_id = league_info.get('previous_league_id')

        # Stop if there is no previous league
        if not league_id:
            break
            
    df_all_years = pd.DataFrame(rows)

    return df_all_years

In [ ]:
def eligibility(df, season: str) -> bool:

    season_int = int(season)
    season_prev = season_int - 1
    season_prev = str(season_prev)

    # Get current season row
    current_row = df[df['season'] == season]
    current_row = current_row.iloc[0]
    current_name = current_row['name']

    # Get previous season row
    prev_row = df[df['season'] == season_prev]
    if prev_row.empty:
        print("b")
        return current_name, season, False  # no previous season, not eligible
    prev_row = prev_row.iloc[0]

    # Check previous season status
    prev_status = prev_row['status']
    if prev_status.lower() != 'complete':
        print("c")
        return current_name, season, False

    # Check total rosters match
    current_total_rosters = current_row['total_rosters']
    prev_total_rosters = prev_row['total_rosters']
    if current_total_rosters != prev_total_rosters:
        print("d")
        return current_name, season, False

    # Check if user IDs match exactly
    current_user_dict = current_row['users']
    current_user_ids = list(current_user_dict.keys())
    prev_user_dict = prev_row['users']
    prev_user_ids = list(prev_user_dict.keys())

    if set(current_user_ids) != set(prev_user_ids):
       return current_name, season, False  # user IDs do not match, exit

    return current_name, season, True

In [342]:
def merge_current_prev(df: pd.DataFrame, season: str):

    season_int = int(season)
    season_prev = season_int - 1
    season_prev = str(season_prev)

    # Current season row
    current_row = df[df['season'] == season]
    if current_row.empty:
        return pd.DataFrame()  # no data
    
    current_row = current_row.iloc[0:1]  # keep as DataFrame

    # Previous season row
    prev_row = df[df['season'] == season_prev]
    if prev_row.empty:
        return current_row  # only current row exists
    
    prev_row = prev_row.iloc[0:1]  # keep as DataFrame

    # Prefix all columns in prev_row
    prev_row_prefixed = prev_row.add_prefix('prev_')

    # Concatenate side by side
    merged_df = pd.concat([current_row.reset_index(drop=True), prev_row_prefixed.reset_index(drop=True)], axis=1)

    # Create 'dosbowl_id' from first 8 chars of both IDs
    merged_df.insert(
        0, 
        'dosbowl_id', 
        merged_df['league_id'].astype(str).str[:8] + merged_df['prev_league_id'].astype(str).str[:8]
    )

    return merged_df

In [ ]:
def league_eligibilities(leagues: list, nfl_season: str):

    results = []
    eligible_leagues = []  # list of DataFrames

    for league_id in leagues:
        df_one_league_all_years = league_info_all_years_df(league_id)

        # 2. Get the latest season for this league
        if df_one_league_all_years.empty:
            continue  # skip if no data

        current_name, season, eligible = eligibility(df_one_league_all_years, nfl_season)

        results.append({
            'league_name': current_name,
            'season': season,
            'eligibility': eligible
        })

            # if eligible, merge and store
        if eligible:
            merged_df = merge_current_prev(df_one_league_all_years, nfl_season)
            eligible_leagues.append(merged_df)

    league_eligibilities = pd.DataFrame(results)


    # Combine all eligible merged dataframes (if any)
    if eligible_leagues:
        eligible_leagues_df = pd.concat(eligible_leagues, ignore_index=True)
    else:
        eligible_leagues_df = pd.DataFrame()  # empty placeholder

    return league_eligibilities, eligible_leagues_df

In [337]:
def get_matchup_df(league_id: str, week: int) -> pd.DataFrame:
    """
    Fetch all matchups for a given league and week from the Sleeper API,
    and return a DataFrame with points, roster_id, matchup_id,
    starters, and starters_points.
    """
    url = f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week}"
    matchups = requests.get(url).json()

    # Fields we want to extract
    fields_to_extract = [
        'roster_id',
        'matchup_id',
        'points',
        'starters',
        'starters_points'
    ]

    # Flatten into DataFrame rows
    rows = []
    for m in matchups:
        flat_data = {}
        for key in fields_to_extract:
            flat_data[key] = m.get(key)
        rows.append(flat_data)

    # Create DataFrame
    df_matchups = pd.DataFrame(rows)

    df_matchups['week'] = week

    return df_matchups

In [365]:
def get_owner_to_roster_map(league_id: str):
    rosters = get_rosters(league_id)  # returns list of dicts like in your example

    # Build mapping: owner_id → roster_id
    mapping = {r['roster_id']: r['owner_id'] for r in rosters if r.get('roster_id') and r.get('owner_id')}

    return mapping

In [377]:
def matchups_df(league_id: str) -> pd.DataFrame:
    all_years_df = league_info_all_years_df(league_id)
    unique_league_ids = list(all_years_df["league_id"].unique())

    all_matchups = []

    for id in unique_league_ids:

        weeks = (all_years_df["settings"][0]['playoff_week_start']) - 1

        season_value = all_years_df.loc[all_years_df['league_id'] == id, 'season'].iloc[0]

        mapping = get_owner_to_roster_map(id)

        # Loop through each week
        for week in range(1, weeks + 1):
            matchup_df = get_matchup_df(id, week)

            matchup_df['owner_id'] = matchup_df['roster_id'].map(mapping)
            owner_col = matchup_df.pop('owner_id')  # remove from current position
            matchup_df.insert(0, 'owner_id', owner_col)  # insert at position 0

            matchup_df['season'] = season_value
            matchup_df['week'] = week

            all_matchups.append(matchup_df)

    # Combine all matchup DataFrames into one master DataFrame
    master_matchups_df = pd.concat(all_matchups, ignore_index=True)
    master_matchups_df.reset_index(drop=True, inplace=True)
    
    return master_matchups_df

## Main Code

#### NFL State

In [300]:
get_nfl_state_df = get_nfl_state()
nfl_week = get_nfl_state_df['leg']
nfl_display_week = get_nfl_state_df['display_week']
nfl_state = get_nfl_state_df['season_type']
nfl_season = get_nfl_state_df['season']

In [114]:
nfl_season, nfl_state, nfl_week, nfl_display_week

('2025', 'regular', 9, 9)

#### Players

In [ ]:
columns_to_keep = [
    'player_id',
    'sport',
    'full_name',
    'fantasy_positions',
    #'status',
    'team',
    'number',
    'birth_date',
    'years_exp',
    #'gsis_id',
    #'espn_id',
    #'kalshi_id'
    #'search_full_name',
    #'depth_chart_order',
]

In [ ]:
df_players = players_df(columns_to_keep)

In [116]:
df_players

,player_id,sport,full_name,fantasy_positions,team,number,birth_date,years_exp
5,8478,nfl,Samuel Womack,[DB],TEN,35.0,1999-07-07,3.0
7,1408,nfl,Le'Veon Bell,[RB],TB,6.0,1992-02-18,9.0
11,2091,nfl,Bashaud Breeland,[DB],ARI,24.0,1992-01-30,8.0
13,11533,nfl,Brandon Aubrey,[K],DAL,17.0,1995-03-14,2.0
17,2064,nfl,DeMarcus Lawrence,[DL],SEA,0.0,1992-04-28,11.0
...,...,...,...,...,...,...,...,...
11358,12693,nfl,Jeremy Crawshaw,[P],DEN,16.0,2001-05-22,0.0
11378,4234,nfl,Noah Brown,[WR],WAS,85.0,1996-01-06,8.0
11387,12845,nfl,Car'lin Vigers,[DB],WAS,22.0,2000-08-16,0.0
11388,5843,nfl,Devin Bush,[LB],CLE,30.0,1998-07-18,6.0


#### User -> Leagues

In [404]:
username = "joelrday"

In [405]:
leagues = list_league_ids(get_user_id(username), nfl_season)

In [406]:
league_eligible, eligible_leagues_df = league_eligibilities(leagues, nfl_season)
league_eligible

,league_name,season,eligibility
0,Dos Bowl,2025,True
1,Bakery Football League,2025,True
2,Bakery Football Franchise Mode,2025,True


#### Once they click!

In [407]:
league_id = leagues[2]

In [408]:
dos_bowl_league_id_df = eligible_leagues_df[eligible_leagues_df['league_id'] == league_id]
dos_bowl_league_id_df

,dosbowl_id,name,status,metadata,settings,avatar,company_id,shard,season,season_type,...,prev_roster_positions,prev_group_id,prev_bracket_id,prev_bracket_overrides_id,prev_loser_bracket_id,prev_loser_bracket_overrides_id,prev_total_rosters,prev_users,prev_reg_season_champ,prev_playoff_data
2,1184597511248311,Bakery Football Franchise Mode,in_season,"{'auto_continue': 'on', 'keeper_deadline': '0'...","{'best_ball': 0, 'last_report': 8, 'waiver_bud...",None,None,392,2025,regular,...,"[QB, RB, RB, FLEX, REC_FLEX, REC_FLEX, REC_FLE...",None,1.172493e+18,None,1.172493e+18,None,10,{'461917932633452544': {'display_name': 'Brand...,865664105695051776,"[{'m': 1, 'r': 1, 'l': 5, 'w': 6, 't1': 5, 't2..."


In [409]:
matchups = matchups_df(league_id)
matchups

,owner_id,roster_id,matchup_id,points,starters,starters_points,week,season
0,865441894052823040,1,4,123.15,"[4881, 8138, 12507, 7564, 11604, 3321, 4217, 4...","[31.45, 21.2, 8.1, 4.6, 15.3, 8.0, 12.5, 8.0, ...",1,2025
1,867945203519094784,2,2,96.10,"[4046, 6813, 7594, 8228, 11624, 5846, 12501, 1...","[28.6, 12.8, 17.9, 13.9, 0.0, 12.3, 3.6, 7.0, ...",1,2025
2,867856435374149632,3,5,122.05,"[6770, 9509, 11584, 8112, 11635, 2449, 6783, 1...","[9.95, 24.4, 14.5, 13.5, 13.4, 11.7, 11.6, 12....",1,2025
3,869024836087918592,4,1,110.20,"[6804, 12527, 9221, 12526, 9493, 7547, 11631, ...","[17.8, 12.0, 15.0, 11.8, 23.1, 8.5, 9.0, 9.0, ...",1,2025
4,461917932633452544,5,3,121.75,"[5849, 9226, 4035, 5967, 8137, 7526, 5947, 509...","[19.95, 16.5, 13.7, 7.9, 6.0, 7.0, 17.7, 10.0,...",1,2025
...,...,...,...,...,...,...,...,...
555,865664105695051776,6,5,107.40,"[3294, 3198, 6130, 8136, 2133, 2216, 1352, 509...","[18.5, 20.5, 5.3, 10.7, 10.1, 8.4, 9.9, 12.0, ...",14,2022
556,862530456044830720,7,2,128.10,"[4046, 4663, 3164, 5846, 1426, 5859, 6786, 49,...","[28.3, 24.4, 17.1, 18.1, 12.9, 17.0, 8.3, 5.0,...",14,2022
557,868235133197639680,8,1,121.85,"[1166, 4866, 4034, 4040, 6801, 5012, 4037, 825...","[29.25, 6.8, 29.3, 22.4, 0.0, 3.7, 10.4, 11.0,...",14,2022
558,868320507634876416,9,2,128.25,"[6904, 4029, 5850, 7066, 7525, 7526, 6824, 523...","[33.55, 8.6, 19.4, 14.8, 17.4, 5.1, 19.4, 11.0...",14,2022


## Weekly Standings

In [ ]:
def get_standings(league_id):
    """Get standings for a league: wins, losses, points by display_name"""
    # Users
    users = {u["user_id"]: u["display_name"] 
             for u in requests.get(f"https://api.sleeper.app/v1/league/{league_id}/users").json()}
    # Rosters (wins, losses, points)
    rosters = requests.get(f"https://api.sleeper.app/v1/league/{league_id}/rosters").json()
    data = []
    for r in rosters:
        uid = r["owner_id"]
        if uid:
            data.append({
                "user": users.get(uid, uid),
                "wins": r["settings"].get("wins", 0),
                "losses": r["settings"].get("losses", 0),
                "points": r["settings"].get("fpts", 0) + r["settings"].get("fpts_decimal", 0)/100
            })
    return pd.DataFrame(data)

In [34]:
seasons = [2024, 2025]

In [35]:
all_dfs = []
for season in seasons:
    leagues = get_league_ids(user_id, season)
    if league_name not in leagues:
        raise ValueError(f"League '{league_name}' not found in {season}")
    league_id = leagues[league_name]
    df = get_standings(league_id)
    df["season"] = season
    all_dfs.append(df)

In [36]:
# Combine & aggregate
combined = pd.concat(all_dfs)
final_df = combined.groupby("user", as_index=False).agg({
    "wins": "sum",
    "losses": "sum",
    "points": "sum"
})

# Save to CSV
final_df.to_csv("sleeper_standings_2024_2025.csv", index=False)

print(final_df)


            user  wins  losses   points
0   BrandynWales    16       7  3282.30
1   JakeReinking    11      12  3019.04
2       Joelrday    11      12  3042.92
3      Martysods    12      11  3132.78
4      Tommypaal     9      14  3099.26
5          Wens8    16       7  3298.96
6         drvuke    14       9  3075.02
7    kentnelson7     6      17  2550.66
8     lukeman781    10      13  3152.26
9  ramseyshaffer    10      13  2895.74


In [37]:
# Combine & aggregate
combined = pd.concat(all_dfs)
final_df = combined.groupby("user", as_index=False).agg({
    "wins": "sum",
    "losses": "sum",
    "points": "sum"
})

# Sort by wins (desc), then points (desc)
final_df = final_df.sort_values(by=["wins", "points"], ascending=[False, False])

# Save to CSV
final_df.to_csv("sleeper_standings_2024_2025.csv", index=False)

print(final_df)


            user  wins  losses   points
5          Wens8    16       7  3298.96
0   BrandynWales    16       7  3282.30
6         drvuke    14       9  3075.02
3      Martysods    12      11  3132.78
2       Joelrday    11      12  3042.92
1   JakeReinking    11      12  3019.04
8     lukeman781    10      13  3152.26
9  ramseyshaffer    10      13  2895.74
4      Tommypaal     9      14  3099.26
7    kentnelson7     6      17  2550.66
